In [1]:
import os
if not os.path.basename(os.getcwd()) in ["ea-nas", "EA-architecture-search"]:
    os.chdir("../")


import sys
import pandas as pd
import LAB.common as fn
import LAB.report_functions as rfn
from src.configuration import Configuration

import plotly
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)

# INTERPRETATION OF EA-NAS RESULTS
This notebook is meant to interpret the results of simulations ran using EA-NAS.
* Analyzes multiple individs
* Groups individs by their versions

Look at the common.py file for more functions to use for analysis

## Load in simulation results: 

Set the correct paths to: 
* Configuraiton file with settings for the simulation
* Path to results save directory

In [2]:
# Configuration file:
config_file = "./LAB/notebook.json"

In [3]:
# Load configuration used:
config = Configuration.from_json(config_file)

# Loads all modules from a given run:
all_modules_grouped = fn.load_all_modules_from_run(f"./results/{config.results_name}")

# Reporting on what modules are loaded: 
print(f"Loaded {sum([len(val) for val in all_modules_grouped.values()])} modules")

Loaded 13 modules


In [4]:
reports = {}
for name, modules in all_modules_grouped.items():
    reports[name] = {
        m.version: fn.progress_report(m.report, m.ID) 
        for m in modules if m.report
    }
print(f"Loaded {sum([len(val) for val in reports.values()])} reports")

Loaded 8 reports


In [5]:
# Modules loaded: 
for key, val in all_modules_grouped.items():
    print(f"\t{key} with versions [v{', v'.join([str(m.version) for m in val])}]")

	Damian with versions [v0]
	Mathias with versions [v0]
	Jelena with versions [v6, v8, v0, v12, v14, v2, v4, v16, v18, v20, v10]


In [6]:
def generate_training_data_plot(module, validation=True, training=False):
    labels = tuple(range(1, len(module.validation_fitness)+1))
    traces = []
    if validation:
        data = tuple(module.validation_fitness)
        traces += [go.Scatter(x=labels, y=data, mode="lines+markers", name=f"{module.ID} Validaiton set")] 
    if training: 
        data = tuple(module.fitness)
        traces += [go.Scatter(x=labels, y=data, mode="lines+markers", name=f"{module.ID} Training set")] 
    return traces

In [7]:
survivers = []
for key, group in all_modules_grouped.items():
    for module in group: # type: Module
        if os.path.isfile(module.relative_save_path(config) + "/model.h5"):
            survivers += [module]

traces = []
for s in survivers: traces += generate_training_data_plot(s)
plotly.offline.iplot({
    "data": traces, 
    "layout": go.Layout(title="Final population + their predecessors")
    
})

In [14]:
fn.create_images(survivers, config)

/Users/magnus/Projects/Python/EA-architecture-search/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6

/Users/magnus/Projects/Python/EA-architecture-search/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6

/Users/magnus/Projects/Python/EA-architecture-search/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6

/Users/magnus/Projects/Python/EA-architecture-search/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6



## Interpreting results per training session: 
Looking at the average gains/losses of performance per class over multiple training sessions below. Gains are positive numbers while losses are negative... 

In [8]:
# Changes over training sessions:
diff_dicts = [
    rfn.score_changes_per_training_session(df) 
    for df in rfn.find_multi_session_reports(reports)
]
pd.concat(diff_dicts).mean()

0, Airplane      0.039370
1, Automobile    0.055892
2, Bird          0.057728
3, Cat           0.014574
4, Deer          0.032677
5, Dog           0.080955
6, Frog          0.043380
7, Horse         0.028085
8, Ship          0.037673
9, Truck         0.053514
micro avg        0.041525
macro avg        0.044385
weighted avg     0.044385
dtype: float64

### Plot of change per training session
Splitting reports by training session number:

In [9]:
multi_session_reports = list(rfn.find_multi_session_reports(reports))

## Looking into training history for a single individ:
* Subject of interst is the specimin 'Galina'
* Contains 5 versions

In [10]:
versions = all_modules_grouped['Jelena']
versions.sort(key=lambda x: x.version)
print([x.ID for x in versions])
v0 = versions[0] # Selecting Galina v0

['Jelena v0', 'Jelena v2', 'Jelena v4', 'Jelena v6', 'Jelena v8', 'Jelena v10', 'Jelena v12', 'Jelena v14', 'Jelena v16', 'Jelena v18', 'Jelena v20']


In [11]:
traces = []
for v in versions: traces += generate_training_data_plot(v)
plotly.offline.iplot({"data": traces })

### Validation accuary plot for transerring leaning

In [12]:
def generate_training_plot_for_transfer_learning(modules):
    traces = []
    prev_end_training = 0
    for i, module in enumerate(tree):
        labels = tuple(range(prev_end_training, len(module.validation_fitness) + prev_end_training))
        trn = module.fitness
        val = module.validation_fitness
        traces += [go.Scatter(x=labels, y=trn, mode="lines+markers", name=f"Training, {module.name} v{i}")]
        traces += [go.Scatter(x=labels, y=val, mode="lines+markers", name=f"Validation, {module.name} v{i}")]
        prev_end_training = len(tree[0].validation_fitness) -1
    return traces


def find_longest_predecessor_range(modules):
    def dive(module):
        res = [module]
        if module.predecessor:
            return dive(module.predecessor) + res
        return res
    _max = 0
    longest_tree = []
    for module in modules:
        tree = dive(module)
        if len(tree) > _max:
            _max = len(tree)
            longest_tree = tree
    
    return longest_tree
    
    
tree = find_longest_predecessor_range(versions)
baseline = go.Scatter(y=tuple([0.82]*31), x=tuple(range(0, 31)), name="Baseline")
plotly.offline.iplot({
    "data": generate_training_plot_for_transfer_learning(tree) + [baseline], 
    "layout": go.Layout()
})

In [13]:
fn.create_images(tree, config)

/Users/magnus/Projects/Python/EA-architecture-search/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6

/Users/magnus/Projects/Python/EA-architecture-search/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6

/Users/magnus/Projects/Python/EA-architecture-search/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6

/Users/magnus/Projects/Python/EA-architecture-search/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6

